# Instrumenting AWS Bedrock client with OpenInference

In this tutorial we will trace model calls to AWS Bedrock using OpenInference. The OpenInference Bedrock tracer instruments the Python `boto3` library, so all `invoke_model` calls will automatically generate traces that can be sent to an OpenTelemetry collector.

ℹ️ This notebook requires a valid AWS configuration and access to AWS Bedrock and the `claude-v2` model from Anthropic.

## 1. Install dependencies and set up OpenTelemetry tracer

First install dependencies

In [ ]:
!pip install boto3 openinference-instrumentation-bedrock

Import libraries

In [1]:
import json

import boto3
from openinference.instrumentation.bedrock import BedrockInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor

Here we're configuring the OpenTelemetry tracer by adding a SpanProcessor. This SpanProcessor will simply print all traces received from OpenInference instrumentation to the console. 

In [2]:
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_console_exporter = ConsoleSpanExporter()
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_console_exporter))
trace_api.set_tracer_provider(tracer_provider=tracer_provider)

## 2. Instrumenting Bedrock clients

Now, let's create a `boto3` session. This initiates a configured environment for interacting with AWS services. If you haven't yet configured `boto3` to use your credentials, please refer to the [official documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/configuration.html). Or, if you have the AWS CLI, run `aws configure` from your terminal.

In [3]:
session = boto3.session.Session()

Clients created using this session configuration are currently uninstrumented. We'll make one for comparison.

In [4]:
uninstrumented_client = session.client("bedrock-runtime")

Now we instrument Bedrock with our OpenInference instrumentor. All Bedrock clients created after this call will automatically produce traces when calling `invoke_model`.

In [5]:
BedrockInstrumentor().instrument()
instrumented_client = session.client("bedrock-runtime")

## 3. Calling the LLM and viewing OpenInference traces

Calling `invoke_model` using the `uninstrumented_client` will produce no traces, but will show the output from the LLM.

In [6]:
prompt = (
    b'{"prompt": "Human: Hello there, how are you? Assistant:", "max_tokens_to_sample": 1024}'
)
response = uninstrumented_client.invoke_model(modelId="anthropic.claude-v2", body=prompt)
response_body = json.loads(response.get("body").read())
print(response_body["completion"])

 I'm doing well, thanks for asking!


LLM calls using the `instrumented_client` will print traces to the console! By configuring the `SpanProcessor` to export to a different OpenTelemetry collector, your OpenInference spans can be collected and analyzed to better understand the behavior of your LLM application.

In [7]:
response = instrumented_client.invoke_model(modelId="anthropic.claude-v2", body=prompt)
response_body = json.loads(response.get("body").read())
print(response_body["completion"])

{
    "name": "bedrock.invoke_model",
    "context": {
        "trace_id": "0x40d2b8f396d1775c0d1616cc2134ebd4",
        "span_id": "0x0a79281232f70dbd",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2024-02-07T16:27:33.541912Z",
    "end_time": "2024-02-07T16:27:34.245463Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "llm.prompts": "Human: Hello there, how are you? Assistant:",
        "llm.invocation_parameters": "{\"max_tokens_to_sample\": 1024}",
        "llm.model_name": "anthropic.claude-v2",
        "message.content": " I'm doing well, thanks for asking!"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {},
        "schema_url": ""
    }
}
 I'm doing well, thanks for asking!
